In [76]:
import pandas as pd
import geopandas as gpd

## Preprocess Election result from Precincts

### Function and declaration

In [ ]:

def process_precinct_votes(precinct_gdf, columns_to_use):
    """
    Args:
        precinct_gdf (pd.DataFrame): The DataFrame containing precinct vote data.
        columns_to_use (list): The list of columns to consider for vote calculations.
    Returns:
        pd.DataFrame: The updated DataFrame with calculated vote totals and lean.
    """
    precinct_gdf['TOT_REP'] = 0
    precinct_gdf['TOT_DEM'] = 0
    precinct_gdf['TOT_OTH'] = 0
    precinct_gdf['TOT_VOT'] = 0
    precinct_gdf['TOT_VOT_DEM_REP'] = 0

    for col in columns_to_use:
        if len(col) >= 7:
            party_code = col[6]
            if party_code == 'R':  # Republican
                precinct_gdf['TOT_REP'] += precinct_gdf[col]
            elif party_code == 'D':  # Democrat
                precinct_gdf['TOT_DEM'] += precinct_gdf[col]
            else:  # Others
                precinct_gdf['TOT_OTH'] += precinct_gdf[col]

    precinct_gdf['TOT_VOT'] = (
        precinct_gdf['TOT_REP'] +
        precinct_gdf['TOT_DEM'] +
        precinct_gdf['TOT_OTH']
    )

    precinct_gdf['TOT_VOT_DEM_REP'] = (
        precinct_gdf['TOT_REP'] +
        precinct_gdf['TOT_DEM']
    )

    precinct_gdf['LEAN'] = precinct_gdf.apply(
        lambda row: 'Unknown' if row['TOT_REP'] == 0 and row['TOT_DEM'] == 0 else
                    ('Republican' if row['TOT_REP'] > row['TOT_DEM'] else
                     ('Democratic' if row['TOT_DEM'] > row['TOT_REP'] else 'Neutral')),
        axis=1
    )

    return precinct_gdf

### Main script

In [77]:
sc_precincts_gdf = gpd.read_file('states/south_carolina/geodata/south_carolina_precincts.geojson')

In [78]:
print(sc_precincts_gdf.columns.values)

['UNIQUE_ID' 'COUNTYFP' 'County' 'NAME' 'Prec_Code' 'G22A1NO' 'G22A1YES'
 'G22A2NO' 'G22A2YES' 'G22AGRCNEL' 'G22AGRGEDM' 'G22AGROWRI' 'G22AGRRWEA'
 'G22ATGOWRI' 'G22ATGRWIL' 'G22COMOWRI' 'G22COMRECK' 'G22GOVDCUN'
 'G22GOVLREE' 'G22GOVOWRI' 'G22GOVRMCM' 'G22SOSDBUT' 'G22SOSOWRI'
 'G22SOSRHAM' 'G22SUPAELL' 'G22SUPDELL' 'G22SUPGMIC' 'G22SUPOWRI'
 'G22SUPRWEA' 'G22TREAWOR' 'G22TREOWRI' 'G22TRERLOF' 'G22USSDMAT'
 'G22USSOWRI' 'G22USSRSCO' 'GCON01AODD' 'GCON01DAND' 'GCON01OWRI'
 'GCON01RMAC' 'GCON02DLAR' 'GCON02OWRI' 'GCON02RWIL' 'GCON03OWRI'
 'GCON03RDUN' 'GCON04OWRI' 'GCON04RTIM' 'GCON05DHUN' 'GCON05GGAI'
 'GCON05OWRI' 'GCON05RNOR' 'GCON06DCLY' 'GCON06OWRI' 'GCON06RBUC'
 'GCON07DSCO' 'GCON07OWRI' 'GCON07RFRY' 'GSL001OWRI' 'GSL001RWHI'
 'GSL002OWRI' 'GSL002RSAN' 'GSL003OWRI' 'GSL003RCAR' 'GSL004OWRI'
 'GSL004RHIO' 'GSL005OWRI' 'GSL005RCOL' 'GSL006OWRI' 'GSL006RCRO'
 'GSL007DSAL' 'GSL007OWRI' 'GSL007RWES' 'GSL008ATOD' 'GSL008DMAC'
 'GSL008OWRI' 'GSL008RCHA' 'GSL009DPOL' 'GSL009OWRI' 'GSL009R

In [79]:
print(sc_precincts_gdf.head())  

                     UNIQUE_ID COUNTYFP     County             NAME Prec_Code  \
0  ABBEVILLE-:-ABBEVILLE NO. 1      001  ABBEVILLE  ABBEVILLE NO. 1       001   
1  ABBEVILLE-:-ABBEVILLE NO. 2      001  ABBEVILLE  ABBEVILLE NO. 2       002   
2  ABBEVILLE-:-ABBEVILLE NO. 3      001  ABBEVILLE  ABBEVILLE NO. 3       003   
3  ABBEVILLE-:-ABBEVILLE NO. 4      001  ABBEVILLE  ABBEVILLE NO. 4       004   
4       ABBEVILLE-:-ANTREVILLE      001  ABBEVILLE       ANTREVILLE       005   

   G22A1NO  G22A1YES  G22A2NO  G22A2YES  G22AGRCNEL  ...  AIA_NHSP22  \
0    355.0     547.0    346.0     554.0        37.0  ...         0.0   
1    254.0     275.0    252.0     274.0        57.0  ...         0.0   
2    219.0     284.0    241.0     279.0        30.0  ...        25.0   
3    197.0     208.0    174.0     242.0        16.0  ...         0.0   
4    280.0     375.0    297.0     375.0        24.0  ...         0.0   

   ASN_NHSP22  HPI_NHSP22  OTH_NHSP22  TOT_REP  TOT_DEM  TOT_OTH  TOT_VOT  \
0  

In [83]:
columns_to_ignore = ['UNIQUE_ID', 'COUNTYFP', 'County', 'NAME', 'Prec_Code',
    'MEDN_INC22', 'TOT_HOUS22', '0_35K', '35K_60K', '60K-100K', 
    '100K_125K', '125K_150K', '150K_MORE', 'TOT_POP22', 
    'NHSP_POP22', 'HSP_POP22', 'WHT_NHSP22', 'BLK_NHSP22', 
    'AIA_NHSP22', 'ASN_NHSP22', 'HPI_NHSP22', 'OTH_NHSP22', 'geometry', 'TOT_REP', 'TOT_DEM', 'TOT_OTH', 'TOT_VOT', 'LEAN'
]

In [84]:
columns_to_use = [col for col in sc_precincts_gdf.columns if col not in columns_to_ignore]

In [85]:
print(columns_to_use)

['G22A1NO', 'G22A1YES', 'G22A2NO', 'G22A2YES', 'G22AGRCNEL', 'G22AGRGEDM', 'G22AGROWRI', 'G22AGRRWEA', 'G22ATGOWRI', 'G22ATGRWIL', 'G22COMOWRI', 'G22COMRECK', 'G22GOVDCUN', 'G22GOVLREE', 'G22GOVOWRI', 'G22GOVRMCM', 'G22SOSDBUT', 'G22SOSOWRI', 'G22SOSRHAM', 'G22SUPAELL', 'G22SUPDELL', 'G22SUPGMIC', 'G22SUPOWRI', 'G22SUPRWEA', 'G22TREAWOR', 'G22TREOWRI', 'G22TRERLOF', 'G22USSDMAT', 'G22USSOWRI', 'G22USSRSCO', 'GCON01AODD', 'GCON01DAND', 'GCON01OWRI', 'GCON01RMAC', 'GCON02DLAR', 'GCON02OWRI', 'GCON02RWIL', 'GCON03OWRI', 'GCON03RDUN', 'GCON04OWRI', 'GCON04RTIM', 'GCON05DHUN', 'GCON05GGAI', 'GCON05OWRI', 'GCON05RNOR', 'GCON06DCLY', 'GCON06OWRI', 'GCON06RBUC', 'GCON07DSCO', 'GCON07OWRI', 'GCON07RFRY', 'GSL001OWRI', 'GSL001RWHI', 'GSL002OWRI', 'GSL002RSAN', 'GSL003OWRI', 'GSL003RCAR', 'GSL004OWRI', 'GSL004RHIO', 'GSL005OWRI', 'GSL005RCOL', 'GSL006OWRI', 'GSL006RCRO', 'GSL007DSAL', 'GSL007OWRI', 'GSL007RWES', 'GSL008ATOD', 'GSL008DMAC', 'GSL008OWRI', 'GSL008RCHA', 'GSL009DPOL', 'GSL009OWRI', '

In [ ]:
sc_precincts_gdf = process_precinct_votes(sc_precincts_gdf, columns_to_use)

In [88]:
print(sc_precincts_gdf[['TOT_REP', 'TOT_DEM', 'TOT_OTH', 'TOT_VOT', 'TOT_VOT_DEM_REP', 'LEAN']].head())

   TOT_REP  TOT_DEM  TOT_OTH  TOT_VOT  TOT_VOT_DEM_REP        LEAN
0   7708.0    984.0   2079.0  10771.0           8692.0  Republican
1   2714.0   1589.0   1354.0   5657.0           4303.0  Republican
2   3748.0    925.0   1216.0   5889.0           4673.0  Republican
3   3348.0    474.0    926.0   4748.0           3822.0  Republican
4   6118.0    493.0   1507.0   8118.0           6611.0  Republican


In [89]:
print(sc_precincts_gdf[['LEAN']].value_counts())

LEAN      
Republican    1911
Democratic     346
Unknown          4
Name: count, dtype: int64


In [90]:
print(sc_precincts_gdf.crs)

EPSG:4326


In [91]:
sc_precincts_gdf.to_file(
    "states/south_carolina/geodata/south_carolina_precincts.geojson",
    driver="GeoJSON",
    drop_crs=True
)

In [93]:
md_precincts_gdf = gpd.read_file('states/maryland/geodata/maryland_precincts.geojson')


In [94]:
print(md_precincts_gdf.columns.values)

['NAME' 'NUMBER' 'JURSCODE' 'VOTESPRE' 'G20PREDBID' 'G20PRERTRU'
 'G20PRELJOR' 'G20PREGHAW' 'G20PREBSEG' 'G20PREOWRI' 'MEDN_INC22'
 'TOT_HOUS22' '0_35K' '35K_60K' '60K-100K' '100K_125K' '125K_150K'
 '150K_MORE' 'TOT_POP22' 'NHSP_POP22' 'HSP_POP22' 'WHT_NHSP22'
 'BLK_NHSP22' 'AIA_NHSP22' 'ASN_NHSP22' 'HPI_NHSP22' 'OTH_NHSP22'
 'geometry']
